In [1]:
# %matplotlib ipympl 
import numpy as np
import pandas as pd
import csv
import datetime
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import boxcox
from scipy.stats import yeojohnson

In [2]:
# load all data into dataframe
def load_data(path, file_names, aliases):
    dates = {}
    for data_set_idx in range(len(data_files)):
        cur_alias = aliases[data_set_idx]
        with open(path + data_files[data_set_idx] + '.csv', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            spamreader.__next__()
            for row in spamreader:
                try:
                    cur_date = datetime.datetime.strptime(row[0], '%m/%d/%Y')

                except Exception as e: 
                   continue
                if not cur_date in dates:
                    dates[cur_date] = {}
#                     # need to generalize here
#                 if data_set_idx == 0 or data_set_idx == 2:
#                     dates[cur_date][cur_alias] = float(row[4])
#                 elif data_set_idx == 1 or data_set_idx == 3 or data_set_idx == 4 or data_set_idx == 5:
                try:
                    dates[cur_date][cur_alias] = float(row[1])
                except:
                    print(row[1])
                    print(cur_alias)
                    print(row)
                    

    df = pd.DataFrame.from_dict(dates, orient='index')
    # df.columns = aliases
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'Date'})
    df = df.sort_values('Date')
    df = df.reset_index(drop=True)
    return df


In [3]:
# not_null = df.query(baseline_asset + ".notnull()")
# not_null.reset_index(drop = True, inplace = True)

In [4]:
def load_div_data(asset_list, file_list):
    div_data = {}
    for asset, file_path in zip(asset_list, file_list):
        # Initialize data structure for the asset
        div_data[asset] = {"payment_date": set(), "ex_date": set(), "amount": {}}
        
        # Read CSV file into DataFrame
        if file_path != None:
            df = pd.read_csv(file_path + ".csv", delimiter=',', header=0)
            
            # Iterate over rows in the DataFrame
            for index, row in df.iterrows():
                # Extract relevant data
                ex_date = row["Ex/EFF Date"]
                cash_amount = row["Cash Amount"]
                payment_date = row["Payment Date"]
                
                # Update div_data with extracted data
                div_data[asset]["ex_date"].add(ex_date)
                div_data[asset]["payment_date"].add(payment_date)
                div_data[asset]["amount"][ex_date] = cash_amount
            
    return div_data


def get_x_days_ret(asset, df, div_data, distance, idx):
    start_idx = 0
    end_idx = 0
    if distance < 0:
        distance = abs(distance)
        start_idx = idx - distance
        end_idx = idx + 1
    else:
        start_idx = idx
        end_idx = idx + distance + 1
        
    num_shares = 1
    dollars = 0
    for i in range(start_idx, end_idx):
        if df.iloc[i]["Date"] in div_data["payment_date"]:
            num_shares += dollars / df.iloc[i][asset]
            dollars = 0
        if df.iloc[i]["Date"] in div_data["ex_date"]:
            dollars += div_data["amount"][df.iloc[i]["Date"]] * num_shares
    final_val = num_shares * df.iloc[end_idx - 1][asset] + dollars
    start_val = df.iloc[start_idx][asset]
    return (final_val - start_val) / start_val
            
            
            
    

def add_correlaries_div(cor_assets, cor_days_out, pred_distance, df, assets, div_data):
    # stores percent changes from past x days 
    cors = [[] for i in range(len(cor_assets))]
    # stores percent changes for x future days for each asset
    futs = {}
    for a in assets:
        futs[a] = []
    
    # iterate through all data points
    for idx, row in df.iterrows():
        # past data points
        for alias_idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
            if idx > days_out: # check for enough data
                # get percent change
                time_period_change = get_x_days_ret(asset, df, div_data[asset], -days_out, idx)  
                cors[alias_idx].append(time_period_change)
            else:
                cors[alias_idx].append(None)
        
        #future data
        for asset in assets:
            cur_price = row[asset]
            if idx + pred_distance < df.shape[0] and not pd.isna(cur_price) and not pd.isna(df.iloc[idx + pred_distance][asset]):
                time_period_change = get_x_days_ret(asset, df, div_data[asset], pred_distance, idx)  
                futs[asset].append(time_period_change) 
            else:
                futs[asset].append(None)
    # input into data frame
    for idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
        name = asset + "_" + str(days_out) + "_dys"
        df.insert(df.shape[1], name, cors[idx], True)
    
    for asset in futs.keys():
        name = asset + "_fut_" + str(pred_distance) + "dys"
        df.insert(df.shape[1], name, futs[asset], True)

In [5]:
# adds correlation metrix to dataframe
def add_correlaries(cor_assets, cor_days_out, pred_distance, df, assets):
    # stores percent changes from past x days 
    cors = [[] for i in range(len(cor_assets))]
    # stores percent changes for x future days for each asset
    futs = {}
    for a in assets:
        futs[a] = []
    
    # iterate through all data points
    for idx, row in df.iterrows():
        # past data points
        for alias_idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
            cur_price = row[asset]
            if idx > days_out: # check for enough data
                # get percent change
                last_time_period = df.loc[idx - days_out - 1].at[asset]
                time_period_change = (cur_price - last_time_period)/last_time_period
                cors[alias_idx].append(time_period_change)           
            else:
                cors[alias_idx].append(None)
        
        #future data
        for asset in assets:
            cur_price = row[asset]
            if idx + pred_distance < df.shape[0] and not pd.isna(cur_price) and not pd.isna(df.iloc[idx + pred_distance].at[asset]):
                fut_val = df.iloc[idx + pred_distance].at[asset]
                time_period_change = (fut_val - cur_price)/cur_price  
                futs[asset].append(time_period_change) 
            else:
                futs[asset].append(None)
    # input into data frame
    for idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
        name = asset + "_" + str(days_out) + "_dys"
        df.insert(df.shape[1], name, cors[idx], True)
    
    for asset in futs.keys():
        name = asset + "_fut_" + str(pred_distance) + "dys"
        df.insert(df.shape[1], name, futs[asset], True)

In [6]:
def add_pred_differences(pred_distance, baseline_asset, assets, df):
    for idx, asset in enumerate(assets):
#       for idx2, asset2 in enumerate(assets[idx + 1: ]): if you want all differences
        if asset != baseline_asset:
            change_asset = df[asset + "_fut_" + str(pred_distance) + "dys"]
            change_baseline = df[baseline_asset + "_fut_" + str(pred_distance) + "dys"]
            diff = change_asset - change_baseline 
            df.insert(df.shape[1], asset + "_" + baseline_asset + "_" + str(pred_distance) + "dys_diff", diff, True)

In [7]:
# not_null.iloc[5700:5750,[0, 7,8, 9, 10,11,12,13,14,15,16,17,18,19,20]]

In [8]:
# not_null.iloc[-160:,0: 10]

In [9]:
# not_null.columns

In [10]:
# #use sklearn.preprocessing.PowerTransformer instead

# plt.figure()

# column_name = 'sp_20_dys'
# column = not_null[column_name] 
# column = column[~np.isnan(column)]
# print(column)
# # column += np.array([1 for i in range(len(column))])
# # print(column)
# plt.figure()
# plt.hist(column , color = 'red', bins = 500, density=True)
# mean = np.mean(column)
# std = np.std(column)
# print(mean)
# print(std)
# x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))


# plt.figure()
# plt.hist(yeojohnson(column)[0] , color = 'red', bins = 500, density=True)
# mean = np.mean(yeojohnson(column)[0])
# std = np.std(yeojohnson(column)[0])
# print(mean)
# print(std)
# x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))
# # plt.hist(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])) , color = 'red', bins = 500, density=True)
# # mean = np.mean(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])))
# # std = np.std(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])))



# # column += np.array([1 for i in range(len(column))])
# # column = np.log(column)
# # plt.figure()
# # plt.hist(yeojohnson(column)[0] , color = 'red', bins = 500, density=True)
# # mean = np.mean(yeojohnson(column)[0])
# # std = np.std(yeojohnson(column)[0])
# # print(mean)
# # print(std)
# # x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))
# # plt.hist(sp_not_null['sp_fut_2wks'], color = 'red', bins = 500)
# # plt.hist(sp_not_null['re_fut_2wks'], color = 'green', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['bnd_fut_2wks'], color = 'blue', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['gld_fut_2wks'], color = 'yellow', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['eu_fut_2wks'], color = 'green', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['jp_fut_2wks'], color = 'blue', bins = 500, alpha = .5,)
# plt.show()

In [11]:
# fig = plt.figure()
# ax = fig.add_subplot(projection = '3d')

# ax.scatter(not_null["sp_last_month"], not_null["re_last_month"], not_null["re_sp_2wk_diff"])
# ax.set_xlabel('sp_last_month')
# ax.set_ylabel('re_last_month')
# ax.set_zlabel('re_sp_2wk_diff')
# plt.show()

In [12]:
def get_rvs(baseline, df, aliases, pred_distance, print_mats=False):
    rvs = {}
    
    valid_cols = []
    for col in df.columns:
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            valid_cols.append(col)

    valid_cols.append(None)
    
    for asset in aliases:
        if asset != baseline:
            valid_cols[-1] = (asset + "_" + baseline + "_" + str(pred_distance) + "dys_diff")

            cov_mat = df[valid_cols]
            # trying out multiplying every times 100 to get larger covariacnce and then maybe wont get the issue with comuns being 
            # linera
            # cov_mat = cov_mat.multiply(100)
            cov_matrix = pd.DataFrame.cov(cov_mat)
            cov_mat = cov_mat.cov()
            cov_mat = cov_mat.to_numpy()
            if print_mats:
                print(asset)
                print(cov_matrix)


            # means of values
            means = []
            for col in valid_cols:
                means.append(np.mean(df[col]))
            if print_mats:
                print(means)

            rv = multivariate_normal(mean=means, cov=cov_mat, allow_singular=True)
            rvs[asset] = rv
    return rvs
        


def predict(asset, baseline, rv, inputs, get_plots=False, do_print=False, get_50_pt = False):
    START = -.25
    STOP = .25
    INCREMENT = .00005

    probs = []
   
    x = np.arange(START, STOP, INCREMENT)
    inputs.append(None)
    for val in x:
        # make an array with all the current values
        # insert past month performance
        #"sp", "re", "bnd", "eu", "jp", "gld", future difference
        inputs[-1] = val
        probs.append(rv.pdf(inputs))


    cdf = []
    for idx in range(x.size - 1):
        cur_prob = probs[idx]
        next_prob = probs[idx + 1]
        rieman_sum = min(cur_prob, next_prob) * INCREMENT
        rieman_sum += max(cur_prob, next_prob) - min(cur_prob, next_prob) * INCREMENT / 2
        if len(cdf) > 0:
            cdf.append(rieman_sum + cdf[-1])
        else:
            cdf.append(rieman_sum)
                                 
    if get_plots:
        fig1 = plt.figure()
        ax = fig1.add_subplot(111)
        plt.title("pdf")
        plt.xlabel("difference between performance of " + asset + " and " + baseline)
        plt.ylabel("probability")
        ax.plot(x, probs/cdf[-1])
        plt.show()

    for idx in range(len(cdf)):
        cdf[idx] /= cdf[-1]

    if get_plots:
        fig2 = plt.figure()
        ax = fig2.add_subplot(111)
        plt.title("cdf")
        plt.xlabel("difference between performance of " + asset + " and " + baseline)
        plt.ylabel("probability")
        ax.plot(x[:-1], cdf)
        plt.show() 
        
    if get_50_pt:
        # find 50% point
        cur_prob = 0
        idx = 0
        while(cur_prob < .5):
            cur_prob = cdf[idx]
            idx += 1
        fiftyfiftypt = x[idx]
        if do_print:
            print("50 50 change to be above or below")
            print(x[idx])


    #find expected value
    expected_value = 0
    for idx in range(len(cdf)):
        if idx == 0:
            expected_value += cdf[idx] * x[idx]
        else:
            cur_prob = cdf[idx - 1]
            next_prob = cdf[idx]
            actual_prob = next_prob - cur_prob
            expected_value += actual_prob * x[idx]
                                 
    if do_print:
        print("Expected Value")
        print(expected_value)
                                 
    if get_50_pt:
        return fiftyfiftypt, expected_value
    else:
        return expected_value

In [13]:
def test_preds(assets, baseline, pred_distance, df_test, df_train):
    rvs = get_rvs(baseline, df_train, assets, pred_distance)
    preds = {}
    actuals = {}
    for asset in assets:
        if asset != baseline:
            preds[asset] = []
            actuals[asset] = []
    
    pred_columns = []
    for col_idx, col in enumerate(df_test.columns):
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            pred_columns.append(col_idx)
            
            
    print('start')
    for idx, row in df_test.iterrows():
        for asset in assets:
            if asset != baseline:
#                 print(asset)
                col_name = asset + "_" + baseline  + "_" + str(pred_distance) + "dys_diff"
                actual = row[col_name]
                actuals[asset].append(actual)
#                 print(actual)
                columns = []
                pred_input = df_test.iloc[idx, pred_columns]
                if not pred_input.isnull().any():
                    prediction = predict(asset, baseline, rvs[asset], pred_input.tolist(), get_plots=False)
                    preds[asset].append(prediction)
                    # print(prediction)
                else:
                    preds[asset].append(None)
#                     print(None)
#                 print("---------")
        if idx % 10== 0:
            print(idx)

    return preds, actuals            

In [14]:
# inputs here

# path = 'C:\\Users\\plant\\
path = ''

baseline_asset = 'sp'

file_SP = 'SPY'  
div_SP = 'SPYDividend'
file_RE = 'IYR'
div_RE = 'IYRDividend'
file_BND = 'isharesBondIndexSince2003'
div_BND = 'USAggBondDividend'
file_EU = 'USD_EURHistoricalData'
div_EU = None
file_JP = 'USD_JPYHistoricalData'
div_JP = None
file_GLD = 'GoldFuturesHistoricalData'
div_GLD = None
file_MID = 'IJH'
div_MID = 'IJHDividend'
file_SML = 'IJR'
div_SML = 'IJRDividend'
file_RUT = 'IWM'
div_RUT = 'IWMDividend'
file_EST = 'EZU'
div_EST = 'EZUDividend'
file_EMR = 'EEM'
div_EMR = 'EEMDividend'
file_JST = 'EWJ'
div_JST = 'EWJDividend'

# data_files = [file_name_SP, file_name_RE, file_name_BND, file_name_EU, file_name_JPY, file_name_GOLD, file_name_RUT]
# aliases = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut']

# # input correlaries
# cor_assets = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut']
# cor_days_out = [20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 252, 252, 252, 252, 252, 252, 252, 60, 60, 60, 60, 60, 60, 60]
# pred_distance = 10
# assets = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut']


data_files = [file_SP, file_RE, file_BND, file_EU, file_JP, file_GLD, file_MID, file_SML, file_RUT, file_EST, file_EMR, file_JST]
div_files = [div_SP, div_RE, div_BND, div_EU, div_JP, div_GLD, div_MID, div_SML, div_RUT, div_EST, div_EMR, div_JST]
aliases = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]

# input correlaries
# cor_assets = ['sp', "re", "bnd", "eu", "jp", "gld", "sp", "re", "bnd", "eu", "jp", "gld", "sp", "re", "bnd", "eu", "jp", "gld",  "sp", "re", "bnd", "eu", "jp", "gld"]
# cor_days_out = [20,   20,    20,   20,   20,    20,   10,   10,   10, 10,   10,   10,   252,  252, 252, 252,  252,   252,    60,   60,    60,    60,    60,   60]

days_out_list = [1, 5, 10, 20, 60, 120, 252]

cor_assets = []
cor_days_out = []

for a in aliases:
    for d in days_out_list:
        cor_assets.append(a)
        cor_days_out.append(d)

# cor_assets = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]
# cor_days_out = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
pred_distance = 10
assets = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]

In [ ]:
df = load_data(path, data_files, aliases)


In [ ]:
df.iloc[1:2]

In [ ]:
df = df.query(baseline_asset + ".notnull()")
df.reset_index(drop = True, inplace = True)

In [ ]:
# add_correlaries(cor_assets, cor_days_out, pred_distance, df, assets)
div_data = load_div_data(aliases, div_files)

In [ ]:
add_correlaries_div(cor_assets, cor_days_out, pred_distance, df, assets, div_data)

In [ ]:
add_pred_differences(pred_distance, baseline_asset, assets, df)

In [ ]:
df.columns

In [ ]:
df.to_csv('AllData.csv', index=False)

In [ ]:
df = df.copy()

In [15]:
df = pd.read_csv('AllData.csv')

In [16]:
df.iloc[3000: 3050, 10 :20]

,est,emr,bnd,sp_1_dys,sp_5_dys,sp_10_dys,sp_20_dys,sp_60_dys,sp_120_dys,sp_252_dys
3000,36.369999,22.064444,102.430000,-0.002070,0.009042,0.009972,0.018335,0.078382,0.073962,0.099434
3001,36.445000,22.178888,102.379997,0.005476,0.014313,0.006729,0.028605,0.066256,0.087401,0.104649
3002,36.345001,22.301111,102.360001,0.001485,0.008057,0.004719,0.029434,0.066058,0.086190,0.091760
3003,36.419998,22.244444,102.570000,-0.001895,0.003729,0.002068,0.015936,0.063477,0.083646,0.089494
3004,36.299999,22.427778,102.400002,-0.002146,0.000828,0.000497,0.012905,0.053976,0.080547,0.086179
3005,36.145000,22.194445,102.400002,-0.004716,-0.001825,0.007200,0.008805,0.060379,0.078730,0.081543
3006,35.595001,21.511110,102.300003,-0.012219,-0.019393,-0.005357,-0.003188,0.056173,0.072473,0.056830
3007,35.375000,21.247778,102.260002,-0.006901,-0.027604,-0.019769,-0.000762,0.044614,0.065938,0.048512
3008,35.474998,21.233334,102.330002,0.005084,-0.020804,-0.017153,-0.001515,0.054030,0.075925,0.050297
3009,35.080002,21.274445,102.300003,-0.001433,-0.020104,-0.019293,-0.006459,0.061861,0.060910,0.044629


In [17]:
df.iloc[8060: 8061, [0, 1, 2, 3, 4, 5, 6, 7, 20]]

,Date,sp,gld,jst,eu,jp,mid,sml,re_1_dys


In [71]:
columns_to_use = []
dont_use = [ "sp_252_dys", "rut_1_dys", "est_1_dys",  "est_5_dys", "est_20_dys"]
# "sp_252_dys",
aliases_not_use = ['mid', 'sml', 'jst', 'est', 'emr', 'gld', 'jp', 'eu']
# rut
time_not_use = [1, 5, 120]
for col in df.columns:
    if col not in dont_use:
        use = True
        for a in aliases_not_use:
            if a in col:
                use = False
        for t in time_not_use:
            if "_" + str(t) + "_" in col:
                use = False
        if use:
            columns_to_use.append(col)
            
aliases_to_use = []
for a in aliases:
    if not a in aliases_not_use:
        aliases_to_use.append(a)

In [79]:
df_test = df[(df['Date'] >= '2008-01-01') & (df['Date'] <= '2008-12-31')][columns_to_use]
df_test = df_test.reset_index(drop=True)
df_train = df[(df['Date'] < '2008-01-01')][columns_to_use]

In [80]:
df_test[0: 20]

,Date,sp,rut,re,bnd,sp_10_dys,sp_20_dys,sp_60_dys,re_10_dys,re_20_dys,...,rut_20_dys,rut_60_dys,rut_252_dys,sp_fut_10dys,re_fut_10dys,bnd_fut_10dys,rut_fut_10dys,re_sp_10dys_diff,bnd_sp_10dys_diff,rut_sp_10dys_diff
0,2008-01-02,144.929993,75.160004,65.150002,101.650002,-0.000965,-0.018621,-0.070067,0.000614,-0.062725,...,-0.007789,-0.106621,-0.036781,-0.054854,-0.075211,0.009838,-0.074375,-0.020357,0.064692,-0.019521
1,2008-01-03,144.860001,74.330002,63.040001,101.919998,-0.006992,-0.010249,-0.065540,-0.044124,-0.068696,...,-0.008272,-0.109820,-0.049002,-0.078904,-0.055838,0.013246,-0.086775,0.023066,0.092149,-0.007871
2,2008-01-04,141.309998,72.089996,61.040001,102.129997,-0.031327,-0.050400,-0.096945,-0.082105,-0.123240,...,-0.056907,-0.143621,-0.080133,-0.065459,-0.033093,0.010966,-0.067554,0.032366,0.076425,-0.002095
3,2008-01-07,141.190002,72.250000,61.610001,102.260002,-0.038215,-0.064595,-0.096210,-0.077004,-0.145729,...,-0.076677,-0.141006,-0.058387,-0.074155,-0.009739,0.011246,-0.076263,0.064417,0.085401,-0.002108
4,2008-01-08,138.910004,70.019997,59.369999,102.519997,-0.062243,-0.079518,-0.106516,-0.125497,-0.172774,...,-0.107912,-0.158009,-0.083868,-0.036354,0.111673,0.005657,-0.014282,0.148027,0.042012,0.022073
5,2008-01-09,140.369995,70.730003,60.540001,102.279999,-0.059371,-0.076999,-0.102092,-0.131919,-0.177110,...,-0.105928,-0.156168,-0.083215,-0.038327,0.068880,0.005475,-0.024883,0.107207,0.043802,0.013444
6,2008-01-10,141.289993,71.699997,61.340000,102.070000,-0.055232,-0.044757,-0.088510,-0.105570,-0.115629,...,-0.060904,-0.134581,-0.077814,-0.058391,0.040267,0.009307,-0.045049,0.098658,0.067698,0.013342
7,2008-01-11,140.149994,70.220001,61.240002,102.510002,-0.050924,-0.061726,-0.088633,-0.078821,-0.119862,...,-0.086985,-0.142613,-0.102390,-0.035034,0.080666,0.006926,-0.004272,0.115700,0.041960,0.030761
8,2008-01-14,141.279999,70.800003,61.070000,102.589996,-0.040869,-0.052194,-0.084084,-0.067634,-0.106903,...,-0.074389,-0.138896,-0.103571,-0.038010,0.077125,0.003022,-0.009887,0.115134,0.041031,0.028123
9,2008-01-15,138.169998,69.500000,59.480000,102.959999,-0.054989,-0.061154,-0.100983,-0.094673,-0.103001,...,-0.073333,-0.152646,-0.114085,-0.023594,0.080027,0.000971,-0.004029,0.103621,0.024565,0.019565


In [81]:
df_test.shape

(253, 27)

In [82]:
preds, actuals = test_preds(aliases_to_use, baseline_asset, pred_distance, df_test, df_train)

start
0
10
20
30


KeyboardInterrupt: 

###### import pickle 

with open('preds.pkl', 'wb') as f:
    pickle.dump(preds, f)

with open('actuals.pkl', 'wb') as f:
    pickle.dump(actuals, f)



In [ ]:
import pickle
with open("preds.pkl",'rb') as f:
    preds = pickle.load(f)
    
with open("actuals.pkl",'rb') as f:
    actuals = pickle.load(f)

In [ ]:
sign_cor_neg = 0
sign_cor_pos = 0
incor_actual_neg = 0
incor_actual_pos = 0
total_diff = 0
total_count = 0
total_correct = 0

for pred, actual in zip(preds['re'], actuals['re']):
    if pred != None and not pd.isna(actual):
        if pred < 0 and actual < 0:
            sign_cor_neg += 1
            total_correct += 1
        elif pred > 0 and actual > 0:
            sign_cor_pos += 1
            total_correct += 1
        elif actual < 0:
            incor_actual_neg += 1
        else:
            incor_actual_pos +=1
        total_count += 1
        total_diff += abs(pred - actual)

In [ ]:
print("pred neg actual neg")
print(sign_cor_neg)
print("pred pos actual pos")
print(sign_cor_pos)
print("pred pos actual neg")
print(incor_actual_neg) 
print("pred neg actual pos")
print(incor_actual_pos )

print("-----")
print("total neg")
print(sign_cor_neg + incor_actual_neg)
print("total pos")
print(sign_cor_pos + incor_actual_pos)
print("----")
print("ave diff")
print(total_diff/total_count)
print("percent correct")
print(total_correct/total_count)


In [78]:

all_asset_total = 0
asset_num = 0

for a in aliases_to_use:
    if a != baseline_asset :
    # and a != "eu" and a != "gld" and a != "jp":
        asset_num += 1
        test_asset = a
        print(test_asset)
        
        asset_total = 0
        
        for i in range(pred_distance):
            offset = i
            count = 0
            total = 1
            period_counts = 0
            for date, pred, actual in zip(df_test["Date"][offset:], preds[test_asset][offset:], actuals[test_asset][offset:]):
                if pred != None and not pd.isna(actual):
                    if count == 10:
                        # if pred > .01:
                        if pred > 0:
                            total *= (1 + 10 * pred * actual)
                            # print("long " + str(pred)[0:7] + " " + str(actual)[0:7]+ ": " + str(date) + ": " + str(total))
                        # if pred < -.01:
                        if pred < 0:
                            total *= (1 + 10 * abs(pred) * -(actual))
                            # print("shorted " + str(pred)[0:7] + " " + str(actual)[0:7] +  ": " + str(date) + ": " + str(total))
                        period_counts += 1
                #         else:
                #             total *= (1 + actual)
                        
                        count = 0
                    count += 1
                
                
            # print(offset)
            # print(total)
            asset_total += total
            # print(period_counts)
        print("average")
        print(asset_total / pred_distance)
        all_asset_total += asset_total / pred_distance
print("all asset average")
print(all_asset_total/ (asset_num))

re
average
1.0009637263382083
bnd
average
0.9869443758502083
rut
average
0.9977022515698215
all asset average
0.995203451252746


##### 